In [1]:
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import random
from pprint import pprint
import matplotlib.pyplot as plt

In [2]:
def check_purity(data):
    
    label_column = data[:, -1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else:
        return False

In [3]:
def classify_data(data):
    
    label_column = data[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

In [4]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):          # excluding the last column which is the label
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        potential_splits[column_index] = unique_values
    
    return potential_splits

In [5]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]

    type_of_feature = FEATURE_TYPES[split_column]
    if type_of_feature == "continuous":
        data_below = data[split_column_values <= split_value]
        data_above = data[split_column_values >  split_value]
    
    # feature is categorical   
    else:
        data_below = data[split_column_values == split_value]
        data_above = data[split_column_values != split_value]
    
    return data_below, data_above

In [6]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts=True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
     
    return entropy

In [7]:
def calculate_overall_entropy(data_below, data_above):
    
    n = len(data_below) + len(data_above)
    p_data_below = len(data_below) / n
    p_data_above = len(data_above) / n

    overall_entropy =  (p_data_below * calculate_entropy(data_below) 
                      + p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [8]:
def determine_best_split(data, potential_splits):
    
    overall_entropy = 9999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column=column_index, split_value=value)
            current_overall_entropy = calculate_overall_entropy(data_below, data_above)

            if current_overall_entropy <= overall_entropy:
                overall_entropy = current_overall_entropy
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

In [9]:
def determine_type_of_feature(df):
    
    feature_types = []
    n_unique_values_treshold = 15
    for feature in df.columns:
        if feature != "label":
            unique_values = df[feature].unique()
            example_value = unique_values[0]

            if (isinstance(example_value, str)) or (len(unique_values) <= n_unique_values_treshold):
                feature_types.append("categorical")
            else:
                feature_types.append("continuous")
    
    return feature_types

In [10]:
def decision_tree_algorithm(df, counter=0):
    
    # data preparations
    if counter == 0:
        global COLUMN_HEADERS, FEATURE_TYPES
        COLUMN_HEADERS = df.columns
        FEATURE_TYPES = determine_type_of_feature(df)
        data = df.values
    else:
        data = df           
    
    # base cases
    if (check_purity(data)):
        classification = classify_data(data)
        return classification    
    # recursive part
    else:    
        counter += 1

        # helper functions 
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits)
        data_below, data_above = split_data(data, split_column, split_value)
        
        # check for empty data
        if len(data_below) == 0 or len(data_above) == 0:
            classification = classify_data(data)
            return classification
        
        # determine question
        feature_name = COLUMN_HEADERS[split_column]
        type_of_feature = FEATURE_TYPES[split_column]
        if type_of_feature == "continuous":
            question = "{} <= {}".format(feature_name, split_value)
            
        # feature is categorical
        else:
            question = "{} = {}".format(feature_name, split_value)
        
        # instantiate sub-tree
        sub_tree = {question: []}
        
        # find answers (recursion) 
        yes_answer = decision_tree_algorithm(data_below, counter)
        no_answer = decision_tree_algorithm(data_above, counter)
        
        # If the answers are the same, then there is no point in asking the qestion.
        # This could happen when the data is classified even though it is not pure
        # yet (min_samples or max_depth base case).
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
        return sub_tree

In [11]:
def train_test_k_fold_split(df, fold):

    indices = df.index.tolist()
    
    low = int((fold/10)*len(indices))
    high = int(((fold+1)/10)*len(indices))
    test_indices=indices[low:high]
    
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

In [12]:
def classify_example(example, tree):
    question = list(tree.keys())[0]
    feature_name, comparison_operator, value = question.split(" ")

    # ask question
    if comparison_operator == "<=":
        if example[feature_name] <= float(value):
            answer = tree[question][0]
        else:
            answer = tree[question][1]
    
    # feature is categorical
    else:
        if str(example[feature_name]) == value:
            answer = tree[question][0]
        else:
            answer = tree[question][1]

    # base case
    if not isinstance(answer, dict):
        return answer
    
    # recursive part
    else:
        residual_tree = answer
        return classify_example(example, residual_tree)

In [53]:
df = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/glass/glass.data')
df.columns = ['f'+str(i) for i in range(len(df.columns))]
df=df.drop(df.columns[0], axis=1)


train_df, test_df = train_test_k_fold_split(df, 1)
tree = decision_tree_algorithm(train_df)


d=np.array(train_df)
X_train = d[:,0:8]
y_train = d[:,9]
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)


correct = 0
correct_sk = 0
for j in range(len(test_df)):
    example = test_df.iloc[j]
    if classify_example(example, tree)==example[9]: correct+=1
    if clf.predict([example[0:8]])==example[9]: correct_sk+=1

print("Youtube accuracy",correct/len(test_df))
print("Sklearn accuracy",correct_sk/len(test_df))
print("\n\n\n")
pprint(tree)

#  print(type(tree))

import json
json.dumps(tree)

Youtube accuracy 0.7619047619047619
Sklearn accuracy 0.7619047619047619




{'f3 <= 2.68': [{'f2 <= 13.78': [{'f4 <= 1.36': [{'f7 <= 8.93': [7.0, 2.0]},
                                                 {'f9 <= 0.0': [5.0,
                                                                {'f9 <= 0.34': [{'f6 <= 0.13': [5.0,
                                                                                                2.0]},
                                                                                5.0]}]}]},
                                 {'f8 <= 0.0': [{'f6 <= 0.0': [6.0,
                                                               {'f9 <= 0.0': [7.0,
                                                                              2.0]}]},
                                                {'f7 <= 5.87': [5.0, 7.0]}]}]},
                {'f4 <= 1.4': [{'f1 <= 1.51694': [{'f1 <= 1.51589': [1.0,
                                                                     {'f5 <= 72.7': [3.0,
 

'{"f3 <= 2.68": [{"f2 <= 13.78": [{"f4 <= 1.36": [{"f7 <= 8.93": [7.0, 2.0]}, {"f9 <= 0.0": [5.0, {"f9 <= 0.34": [{"f6 <= 0.13": [5.0, 2.0]}, 5.0]}]}]}, {"f8 <= 0.0": [{"f6 <= 0.0": [6.0, {"f9 <= 0.0": [7.0, 2.0]}]}, {"f7 <= 5.87": [5.0, 7.0]}]}]}, {"f4 <= 1.4": [{"f1 <= 1.51694": [{"f1 <= 1.51589": [1.0, {"f5 <= 72.7": [3.0, {"f5 <= 72.88": [2.0, {"f9 <= 0.0": [3.0, 2.0]}]}]}]}, {"f6 <= 0.23": [{"f7 <= 10.17": [{"f5 <= 72.64": [{"f2 <= 13.99": [1.0, {"f2 <= 14.32": [{"f7 <= 9.65": [3.0, 1.0]}, 1.0]}]}, 3.0]}, 2.0]}, {"f3 <= 3.74": [{"f4 <= 1.16": [{"f9 <= 0.0": [{"f6 <= 0.54": [2.0, 1.0]}, 2.0]}, {"f9 <= 0.3": [{"f1 <= 1.51926": [1.0, 7.0]}, 2.0]}]}, 2.0]}]}]}, {"f3 <= 3.45": [{"f5 <= 72.81": [{"f8 <= 0.0": [{"f7 <= 8.6": [{"f9 <= 0.0": [2.0, 3.0]}, 3.0]}, 7.0]}, 2.0]}, {"f9 <= 0.15": [{"f6 <= 0.54": [{"f5 <= 72.72": [2.0, {"f6 <= 0.38": [2.0, 1.0]}]}, 2.0]}, {"f2 <= 12.82": [1.0, 2.0]}]}]}]}]}'